In [ ]:
from keras.layers import Dense,Dropout, Input
from keras.models import Model
import pandas as pd 


In [ ]:
# Function to calculate NMSE Error. 
def Calculate_NMSE(real,prediction):
    difference = real-prediction
    NMSE_error = Squared_err(difference)/Squared_err(real)
    return NMSE_error

def Squared_err(mat):
    Norm = mat[:,0]**2 + 2.*mat[:,1]**2 + 2.*mat[:,2]**2 + 2.*mat[:,3]**2 + 2.*mat[:,4]**2 + 2.*mat[:,5]**2 + mat[:,6]**2
    + 2.*mat[:,7]**2 + 2.*mat[:,8]**2 + 2.*mat[:,9]**2 + 2.*mat[:,10]**2 + mat[:,11]**2 + 2.*mat[:,12]**2 + 2.*mat[:,13]**2
    + 2.*mat[:,14]**2 + mat[:,15]**2 + 2.*mat[:,16]**2 + 2.*mat[:,17]**2 + mat[:,18]**2 + 2.*mat[:,19]**2 + mat[:,20]**2
    return Norm

In [ ]:
# Normlization range [0,1], based on the maximum and minimum of each coloumn. 
def normalize(df,orginal):
    result = df.copy()
    for feature_name in df.columns:
        max_value = orginal[feature_name].max()
        min_value = orginal[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

# Unnormalizatiton Function 
def unnormalize(df,orginal):
    result = df.copy()
    for feature_name in df.columns:
        max_value = orginal[feature_name].max()
        min_value = orginal[feature_name].min()
        result[feature_name] = (df[feature_name])*(max_value - min_value) + min_value 
    return result


In [ ]:
# Data Reading / only portion of the data was provided for testing purposes. 
Data = pd.read_csv("..../Testing_Data.csv", header = None,sep='\,+', dtype= 'float32')
max_min_values = pd.read_csv("..../max_min_data.csv", header = None,sep='\,+', dtype= 'float32')

In [ ]:
# split the design parameter data
Design_parameter = Data.iloc[:,[0,1,2,3,4,50,51,52,53,54,55]]
Design_parameter_max_min = max_min_values.iloc[:,[0,1,2,3,4,50,51,52,53,54,55]]

# Split the stiffness components data 
Stiffness_components = Data.iloc[:,29:50]  
Stiffness_components_max_min = max_min_values.iloc[:,29:50]

# Change the DataFrame type into ndarray
Design_parameter = Design_parameter.values
Stiffness_components = Stiffness_components.values
Design_parameter_max_min = Design_parameter_max_min.values
Stiffness_components_max_min = Stiffness_components_max_min.values


In [ ]:
# Apply normaliztion on the stiffness data. 
Stiffness_components_norm = normalize (pd.DataFrame(Stiffness_components),pd.DataFrame(Stiffness_components_max_min))
Stiffness_components_norm = Stiffness_components_norm.values

In [ ]:
# Build a Forward Model 

input_layer = Input((11,))

## create an MLP architecture with dense layers 
dense_layer1 = Dense(units=512,activation='relu')(input_layer)
dense_layer1 = Dropout(0.05)(dense_layer1)
dense_layer2 = Dense(units=128, activation='relu')(dense_layer1)
dense_layer2 = Dropout(0.05)(dense_layer2)
dense_layer3 = Dense(units=32, activation='relu')(dense_layer2)
dense_layer3 = Dropout(0.05)(dense_layer3)
output_layer = Dense(units=21,activation='sigmoid')(dense_layer3)

## define the model with input layer and output layer
Forward_Model = Model(inputs=input_layer, outputs=output_layer)
Forward_Model.summary()

# Compile the model 
Forward_Model.compile(loss='mse', optimizer='adam')
# Load the weights for the trained model
Forward_Model.load_weights('models/Forward_Model_rotation.h5')

In [ ]:
# Build a generator. 
input_layer_1 = Input((21,))

## create an MLP architecture with dense layers 
a = Dense(units=512,activation = 'relu')(input_layer_1)
a = Dropout(0.05)(a)
a = Dense(units=512,activation = 'relu')(a)
a = Dropout(0.05)(a)
a = Dense(units=512,activation = 'relu')(a)
a = Dropout(0.05)(a)
a = Dense(units=512,activation = 'relu')(a)
output_layer_1 = Dense(units=11,activation = 'sigmoid')(a)

## define the model with input layer and output layer
generator_Model = Model(inputs=input_layer_1, outputs=output_layer_1)
generator_Model.summary()

# Compile the model. 
generator_Model.compile(loss='mse', optimizer='adam')
# Load the weights for the trained model
generator_Model.load_weights("models/generator_weights_rotation.h5") 

In [ ]:
# prediction of the normalized data
Predicted_Design_parameters_norm = generator_Model.predict(Stiffness_components_norm)
Predicted_stiffness_components_norm = Forward_Model.predict(Predicted_Design_parameters_norm)
# Unnormalize the data 
Predicted_Design_parameters = unnormalize(pd.DataFrame(Predicted_Design_parameters_norm),pd.DataFrame(Design_parameter_max_min))
Predicted_stiffness_components = unnormalize(pd.DataFrame(Predicted_stiffness_components_norm),pd.DataFrame(Stiffness_components_max_min))